In [74]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import re
import unidecode
import nltk
from nltk.corpus import stopwords

In [75]:
#load "data copy" df
df = pd.read_csv('data copy.csv')
#drop first item
df = df.drop(df.index[0])

In [76]:
# turn "Data" (xx/xx/xxxx string) and "Hora" (xxhxx string) into a single datetime column and set as index
df['Data'] = pd.to_datetime(df['Data'] + ' ' + df['Hora'], format='%d/%m/%Y %Hh%M')
df = df.set_index('Data')
df.drop(['Hora'], axis=1, inplace=True)
df.head()

,Sessão,Fase,URL_Discurso,Discurso,Orador,Publicação
Data,,,,,,
2000-10-10 10:24:00,179.2.51.O,HOMENAGEM,https://www.camara.leg.br/internet/sitaqweb/Te...,O SR. EDIR OLIVEIRA (PTB-RS. Pronuncia o segui...,"EDIR OLIVEIRA, PTB-RS",DCD11/10/2000 PAG. 50369
2000-10-10 10:42:00,179.2.51.O,HOMENAGEM,https://www.camara.leg.br/internet/sitaqweb/Te...,O SR. PRESIDENTE (Severino Cavalcanti) – Com s...,"SEVERINO CAVALCANTI (PRESIDENTE), PPB-PE",DCD11/10/2000 PAG. 50371
2000-10-10 10:56:00,179.2.51.O,HOMENAGEM,https://www.camara.leg.br/internet/sitaqweb/Te...,O SR. PRESIDENTE (Severino Cavalcanti) – Exmo....,"SEVERINO CAVALCANTI (PRESIDENTE), PPB-PE",DCD11/10/2000 PAG. 50373
2000-10-10 14:20:00,180.2.51.O,PEQUENO EXPEDIENTE,https://www.camara.leg.br/internet/sitaqweb/Te...,O SR. GILMAR MACHADO (PT-MG. Sem revisão do or...,"GILMAR MACHADO, PT-MG",DCD11/10/2000 PAG. 50375
2000-10-10 14:32:00,180.2.51.O,PEQUENO EXPEDIENTE,https://www.camara.leg.br/internet/sitaqweb/Te...,O SR. JOSÉ ANTONIO ALMEIDA (Bloco/PSB-MA. Sem ...,"JOSÉ ANTONIO ALMEIDA, PSB-MA",DCD11/10/2000 PAG. 50377


In [77]:
#drop rows where "Orador" does not contain a comma
df = df[df['Orador'].str.contains(",")]

In [78]:
df['Partido'] = df['Orador'].str.split(',').str[1]
df['Orador'] = df['Orador'].str.split(',').str[0]
df['Estado'] = df['Partido'].str.split('-').str[1]
df['Partido'] = df['Partido'].str.split('-').str[0]
df['Discurso_Original'] = df['Discurso']

for i in range(5):
    print(df['Discurso'][i])

O SR. EDIR OLIVEIRA (PTB-RS. Pronuncia o seguinte discurso.) - Sr. Presidente, Deputado Severino Cavalcanti, Exmo. Sr. Ministro Paulo Costa Leite, Presidente do Superior Tribunal de Justiça, Ilmo. Sr. José Pinto Mota Filho, representante do Governo do Estado do Rio Grande do Sul, Ilmo. Sr. jornalista Fábio Marçal, representante do Correio do Povo, Sras. e Srs. Deputados, especialmente o recém-eleito Prefeito de Uruguaiana, Deputado Caio Riela e sua esposa, senhoras e senhores convidados, prezados colaboradores, funcionários desta Casa, a sessão solene de hoje foi originalmente requerida pelo Deputado Caio Riela e, com a licença de S. Exa., em função da candidatura à Prefeitura do Município de Uruguaiana, coube a nós esta honrosa distinção de homenagear dois jornais de grande importância para o Rio Grande do Sul e para o Brasil.   	O dia 1º de outubro festejou o 105º aniversário do Correio do Povo e o 118º de a Gazeta de Alegrete.  A Gazeta de Alegrete, fundada em 1º de outubro de 1876,

C:\Users\arthur.nery\AppData\Local\Temp\ipykernel_33784\3385256862.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df['Discurso'][i])


In [79]:
df['Orador'] = df['Orador'].apply(lambda x: re.sub(r"\(.*\)", "", x) if isinstance(x, str) else x)

remove = [
    "Sr",
    "Sra",
    "Srª",
    "Srº",
    "Presidente",
    "Presidenta",
    "Sras",
    "Sres",
    "Srs",
    "Exmo",
    "Exma",
    "V Exa",
    "Deputados",
    "S Exa",
    "Exa",
    "º",
    "ª",
]

remove_pattern = r'\b(?:' + '|'.join(map(re.escape, remove)) + r')\b'
df['Discurso'] = df['Discurso'].str.replace(remove_pattern, '', regex=True)

patterns = [r"\([^)]*\)|[A-Z]{2,}|[^\w\s]|\d+", r".*–", r"\s{2,}"]
for pattern in patterns:
    df['Discurso'] = df['Discurso'].apply(lambda x: re.sub(pattern, ' ', x) if isinstance(x, str) else x)

for i in range(5):
    print(df['Discurso'][i])

O Deputado Severino Cavalcanti Ministro Paulo Costa Leite do Superior Tribunal de Justiça Ilmo José Pinto Mota Filho representante do Governo do Estado do Rio Grande do Sul Ilmo jornalista Fábio Marçal representante do Correio do Povo e especialmente o recém eleito Prefeito de Uruguaiana Deputado Caio Riela e sua esposa senhoras e senhores convidados prezados colaboradores funcionários desta Casa a sessão solene de hoje foi originalmente requerida pelo Deputado Caio Riela e com a licença de S em função da candidatura à Prefeitura do Município de Uruguaiana coube a nós esta honrosa distinção de homenagear dois jornais de grande importância para o Rio Grande do Sul e para o Brasil O dia º de outubro festejou o º aniversário do Correio do Povo e o º de a Gazeta de Alegrete A Gazeta de Alegrete fundada em º de outubro de é o mais antigo jornal em circulação no Estado do Rio Grande do Sul e o oitavo em antigüidade no Brasil É também uma das cinco publicações mais antigas do mundo editadas e

C:\Users\arthur.nery\AppData\Local\Temp\ipykernel_33784\2611469559.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df['Discurso'][i])


In [80]:
# lowercasing
df['Discurso'] = df['Discurso'].apply(lambda x: x.lower() if isinstance(x, str) else x)
# remove accents
df['Discurso'] = df['Discurso'].apply(lambda x: unidecode.unidecode(x) if isinstance(x, str) else x)
for i in range(5):
    print(df['Discurso'][i])

o deputado severino cavalcanti ministro paulo costa leite do superior tribunal de justica ilmo jose pinto mota filho representante do governo do estado do rio grande do sul ilmo jornalista fabio marcal representante do correio do povo e especialmente o recem eleito prefeito de uruguaiana deputado caio riela e sua esposa senhoras e senhores convidados prezados colaboradores funcionarios desta casa a sessao solene de hoje foi originalmente requerida pelo deputado caio riela e com a licenca de s em funcao da candidatura a prefeitura do municipio de uruguaiana coube a nos esta honrosa distincao de homenagear dois jornais de grande importancia para o rio grande do sul e para o brasil o dia o de outubro festejou o o aniversario do correio do povo e o o de a gazeta de alegrete a gazeta de alegrete fundada em o de outubro de e o mais antigo jornal em circulacao no estado do rio grande do sul e o oitavo em antiguidade no brasil e tambem uma das cinco publicacoes mais antigas do mundo editadas e

C:\Users\arthur.nery\AppData\Local\Temp\ipykernel_33784\3367473474.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df['Discurso'][i])


In [81]:
nltk.download('stopwords')
stop = stopwords.words('portuguese')
df['Discurso'] = df['Discurso'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop) and len(word) > 2]) if isinstance(x, str) else x)
for i in range(5):
    print(df['Discurso'][i])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arthur.nery\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


deputado severino cavalcanti ministro paulo costa leite superior tribunal justica ilmo jose pinto mota filho representante governo estado rio grande sul ilmo jornalista fabio marcal representante correio povo especialmente recem eleito prefeito uruguaiana deputado caio riela esposa senhoras senhores convidados prezados colaboradores funcionarios desta casa sessao solene hoje originalmente requerida deputado caio riela licenca funcao candidatura prefeitura municipio uruguaiana coube honrosa distincao homenagear dois jornais grande importancia rio grande sul brasil dia outubro festejou aniversario correio povo gazeta alegrete gazeta alegrete fundada outubro antigo jornal circulacao estado rio grande sul oitavo antiguidade brasil tambem cinco publicacoes antigas mundo editadas lingua portuguesa fundador luiz freitas valle barao ibirocay primeiro editor engenheiro militar jesuino melchiades souza familia prunes jose celestino teodonio cirino jose fredolino lourenco sobrinho mario lourenco 

C:\Users\arthur.nery\AppData\Local\Temp\ipykernel_33784\1160792643.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df['Discurso'][i])


In [82]:
nltk.download('rslp')
stemmer = nltk.stem.RSLPStemmer()
df['Discurso'] = df['Discurso'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]) if isinstance(x, str) else x)
for i in range(5):
    print(df['Discurso'][i])

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\arthur.nery\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


deput severin cavalcant ministr paul cost leit superi tribun jus ilm jos pint mot filh represent govern est rio grand sul ilm jornal fabi marc represent correi pov espec rec eleit prefeit uruguai deput cai riel esp senh senh convid prez colabor funcionari dest cas sessa solen hoj orig requer deput cai riel licenc funca candidat prefeit municipi uruguai coub honr distinca homenag doi jorn grand importanc rio grand sul brasil dia outubr festej aniversari correi pov gazet alegret gazet alegret fund outubr antig jorn circulaca est rio grand sul oitav antigu brasil tamb cinc publicaco antig mund edit lingu portugu fund luiz freit vall bara ibirocay prim edit engenh milit jesuin melchiad souz famil prun jos celestin teodoni cirin jos fredolin lourenc sobr mari lourenc prun dirig jorn ano ate dia atual dire advog jornal hei galant ireca jorn tamb integr dire jornal samuel marqu silv heli riociard sant fund bara ibirocay emerit denod vult alegret fim secul pass torn personal destac imp geraco 

C:\Users\arthur.nery\AppData\Local\Temp\ipykernel_33784\799002906.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df['Discurso'][i])


In [83]:
# dropna on "Discurso"
df = df.dropna(subset=['Discurso'])

In [84]:
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = vectorizer.fit_transform(df["Discurso"])
lda = LatentDirichletAllocation(
    n_components=100,
    random_state=0,
    max_iter=10,
    verbose=1,
)
lda.fit(dtm)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(n_components=100, random_state=0, verbose=1)

In [87]:
#show results

for index, topic in enumerate(lda.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    # print original word instead of lemmatized
    
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['tamb', 'publ', 'civil', 'violenc', 'tod', 'distrit', 'crim', 'brasil', 'est', 'milit', 'govern', 'nao', 'seguranc', 'feder', 'polic']


THE TOP 15 WORDS FOR TOPIC #1
['nest', 'dia', 'tod', 'legisl', 'vot', 'execu', 'govern', 'sen', 'nacion', 'cas', 'pod', 'congress', 'nao', 'provis', 'med']


THE TOP 15 WORDS FOR TOPIC #2
['cobranc', 'ganh', 'cobr', 'empr', 'nao', 'pag', 'feder', 'concession', 'real', 'tabel', 'telefon', 'pedagi', 'tarif', 'rend', 'impost']


THE TOP 15 WORDS FOR TOPIC #3
['anal', 'notic', 'public', 'rio', 'artig', 'prim', 'paul', 'grand', 'brasil', 'ano', 'regia', 'imprens', 'diari', 'jornal', 'jorn']


THE TOP 15 WORDS FOR TOPIC #4
['cam', 'dev', 'sociedad', 'sistem', 'nao', 'publ', 'pal', 'congress', 'represent', 'pod', 'tod', 'brasil', 'reform', 'nacion', 'poli']


THE TOP 15 WORDS FOR TOPIC #5
['trabalh', 'quer', 'grand', 'jos', 'prefeit', 'mil', 'desenvolv', 'brasil', 'ano', 'regia', 'cidad', 'pret', 'sao', 'rio', 'tod']


THE TOP 

In [88]:
# add topic column to df
topic_results = lda.transform(dtm)
df['Topico'] = topic_results.argmax(axis=1)
df['Topico'] = df['Topico'].apply(lambda x: x + 1)
df.head()

,Sessão,Fase,URL_Discurso,Discurso,Orador,Publicação,Partido,Estado,Discurso_Original,Topico
Data,,,,,,,,,,
2000-10-10 10:24:00,179.2.51.O,HOMENAGEM,https://www.camara.leg.br/internet/sitaqweb/Te...,deput severin cavalcant ministr paul cost leit...,EDIR OLIVEIRA,DCD11/10/2000 PAG. 50369,PTB,RS,O SR. EDIR OLIVEIRA (PTB-RS. Pronuncia o segui...,60
2000-10-10 10:42:00,179.2.51.O,HOMENAGEM,https://www.camara.leg.br/internet/sitaqweb/Te...,satisfaca registr presenc prof doral galess hi...,SEVERINO CAVALCANTI,DCD11/10/2000 PAG. 50371,PPB,PE,O SR. PRESIDENTE (Severino Cavalcanti) – Com s...,9
2000-10-10 10:56:00,179.2.51.O,HOMENAGEM,https://www.camara.leg.br/internet/sitaqweb/Te...,ministr paul cost leit jos pint fabi marc herm...,SEVERINO CAVALCANTI,DCD11/10/2000 PAG. 50373,PPB,PE,O SR. PRESIDENTE (Severino Cavalcanti) – Exmo....,60
2000-10-10 14:20:00,180.2.51.O,PEQUENO EXPEDIENTE,https://www.camara.leg.br/internet/sitaqweb/Te...,vez ocup tribun abord tem educaca examin orc u...,GILMAR MACHADO,DCD11/10/2000 PAG. 50375,PT,MG,O SR. GILMAR MACHADO (PT-MG. Sem revisão do or...,94
2000-10-10 14:32:00,180.2.51.O,PEQUENO EXPEDIENTE,https://www.camara.leg.br/internet/sitaqweb/Te...,numer divulg ult pesquis institut pesquis econ...,JOSÉ ANTONIO ALMEIDA,DCD11/10/2000 PAG. 50377,PSB,MA,O SR. JOSÉ ANTONIO ALMEIDA (Bloco/PSB-MA. Sem ...,19


In [91]:
# print all "Discurso_Original" where Topico is 30
for column in df[df['Topico'] == 29]['Discurso_Original']:
    print(column)


O SR. DARCÍSIO PERONDI (PMDB-RS. Pela ordem. Sem revisão do orador.) - Sr. Presidente, assomo à tribuna para, rapidamente, fazer duas observações, a primeira delas se refere ao pronunciamento do colega que acaba de se manifestar no sentido de que hoje Brasília vive uma situação caótica.    S.Exa. foi feliz quando, de forma tranqüila e organizada, citou todos os segmentos funcionais que, estando em greve, estão provocando esse caos, pois todos eles são ligados à Central Única dos Trabalhadores, braço sindical do Partido dos Trabalhadores, que quer parar Brasília. 	Apesar de todas as dificuldades, o Governo Roriz está fazendo, está construindo. Tenho ido aos supermercados, às feiras, tenho caminhado e perguntado à população sua opinião: ela está cansada. Quando o braço sindical do Partido dos Trabalhadores impede que uma mãe interne seu filho com pneumonia num hospital público de Brasília, o movimento grevista ultrapassa todos os limites. Pode haver o direito de greve, mas desde que ele 